# VacationPy
----

#### Note

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file = pd.read_csv("../output_data/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudy,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,São Filipe,10,CV,1635042707,68,14.8961,-24.4956,78.53,9.10
1,1,Iskateley,74,RU,1635042708,91,67.6803,53.1512,25.11,7.99
2,2,Hervey Bay,99,AU,1635042708,78,-25.2986,152.8535,89.71,16.60
3,3,Yellowknife,75,CA,1635042709,80,62.4560,-114.3525,41.56,17.27
4,4,New Norfolk,100,AU,1635042680,51,-42.7826,147.0587,55.65,5.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [7]:
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
temp_file=file.loc[(file['Max Temp']>70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudy']==0)]
ideal_weather.head()

,City ID,City,Cloudy,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
153,153,Garhi Khairo,0,PK,1635042756,41,28.0606,67.9803,75.34,4.07
235,235,Salalah,0,OM,1635042568,68,17.0151,54.0924,73.49,5.75
324,324,Karrāpur,0,IN,1635042806,54,23.9500,78.8500,73.04,2.26
349,349,Ibrā’,0,OM,1635042812,70,22.6906,58.5334,73.56,1.10
401,401,Doha,0,QA,1635042440,67,25.2867,51.5333,76.98,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

params = { 
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

#Using the lat/lbg we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
        
    params["location"] = f"{lat},{lng}"
        
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 28.2787314,
                    "lng": 68.4386712
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.28001137989272,
                        "lng": 68.43991832989273
                    },
                    "southwest": {
                        "lat": 28.27731172010728,
                        "lng": 68.43721867010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png",
            "icon_background_color": "#FF9E67",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet",
            "name": "Hotel Al-Harmain",
            "opening_hours": {
                "open_now": true
       

,City ID,City,Cloudy,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,153,Garhi Khairo,0,PK,1635042756,41,28.0606,67.9803,75.34,4.07,Hotel Al-Harmain
1,235,Salalah,0,OM,1635042568,68,17.0151,54.0924,73.49,5.75,Fanar Hotel and Residences
2,324,Karrāpur,0,IN,1635042806,54,23.9500,78.8500,73.04,2.26,Hotel The Crown Palace
3,349,Ibrā’,0,OM,1635042812,70,22.6906,58.5334,73.56,1.10,معسكر ليالي الصحراء Desert Nights Camp
4,401,Doha,0,QA,1635042440,67,25.2867,51.5333,76.98,4.61,Four Seasons Hotel Doha
5,415,Umluj,0,SA,1635042833,60,25.0213,37.2685,76.14,9.51,Juman Hotel
6,529,Alot,0,IN,1635042866,68,23.7667,75.5500,71.38,5.55,"MPT Highway Treat, Mandsaur"


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))